In [59]:
from typing import TypedDict, Dict, Any
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

# Initialize memory (stores last 5 exchanges)
memory = ConversationBufferWindowMemory(k=5, return_messages=True)


In [60]:
from langchain.chat_models import ChatOpenAI

# Initialize the GROQ LLM using OpenAI-compatible endpoint
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key="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",  # replace with your GROQ API key
    model="llama3-70b-8192",      # or another supported model like mixtral
)

In [61]:
class State(TypedDict):
    user_problem: str
    specialist_agent: str
    specialist_response: str
    response: str
    memory: Dict[str, Any]  # Track conversation history

In [62]:
# Problem Classification with memory
def problem_classification(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        """Conversation History: {history}
        
        Analyze this user query and classify it:
        'billing' - payments, invoices, charges, refunds
        'tech' - login problems, technical issues
        'shipping' - orders, deliveries, shipments
        
        User query: {user_problem}
        Classification:"""
    )
    
    chain = prompt | llm
    classification = chain.invoke({
        "user_problem": state["user_problem"],
        "history": state["memory"].get("history", "")
    }).content
    
    return {"specialist_agent": classification}

In [63]:
# Specialist agents with memory
def specialist_template(agent_type: str,agent_prompt: str) -> ChatPromptTemplate:
    return ChatPromptTemplate.from_template(
        f"""Conversation History: {{history}}
        You are a {agent_type} support specialist. Previous messages in this conversation:
        {{{{memory}}}}
        
        User's current issue: {{user_problem}}
        {agent_prompt}
        
        Provide detailed assistance:"""
    )

In [64]:
def billing_agent(state: State) -> State:
    agent_prompt = ChatPromptTemplate.from_template(
        """      
        - Understand the user's concern based on their message and any provided context.
        - Identify if the issue is related to charges, refunds, invoices, subscriptions, or payment methods.
        - Simulate a helpful and professional response to resolve the issue.
        - If the issue cannot be resolved automatically, suggest the next best step (e.g., contact support or escalate).
        """
        """
        - When responding:
            Be clear and concise.
            Maintain a polite and professional tone.
            Include relevant details if available (e.g., refund timelines, invoice numbers, etc.).
        """
    )
    prompt = specialist_template("billing",agent_prompt)

    chain = prompt | llm
    response = chain.invoke({
        "user_problem": state["user_problem"],
        "history": state["memory"].get("history", "")
    }).content
    return {"specialist_response": response}

In [65]:
def tech_agent(state: State) -> State:
    agent_prompt = ChatPromptTemplate.from_template(        """      
        A)  You are a customer support agent specializing in technical issues. The user's query has already been classified as related to technical support.
        B)  Your task is to:
            -  Understand the user's problem based on their message and any provided context.
            -  Identify if the issue involves login problems, account access, errors, bugs, system crashes, or other technical difficulties.
            -  Simulate a helpful and professional response that guides the user toward resolving the issue.
            -  If the issue cannot be resolved automatically, suggest the next best step (e.g., clear instructions for troubleshooting, or escalation to human support).
        """
        """
        C)  When responding:
                - Be clear, specific, and actionable.
                - Maintain a polite and empathetic tone.
                - Use plain language that non-technical users can understand.
        """
)
    prompt = specialist_template("technical",agent_prompt)
    chain = prompt | llm
    response = chain.invoke({
        "user_problem": state["user_problem"],
        "history": state["memory"].get("history", "")
    }).content
    return {"specialist_response": response}

In [66]:
def shipping_agent(state: State) -> State:
    agent_prompt = ChatPromptTemplate.from_template(   """      
        A) You are a customer support agent specializing in shipping and delivery-related issues. The user's query has already been categorized as related to shipping.
        B) Your task is to:
            - Understand the user's concern based on their message and any provided context.
            - Identify if the issue involves order tracking, delayed delivery, shipment status, address updates, or missing packages.
            - Simulate a helpful and professional response that provides the latest information or suggests next steps.
            - If the issue cannot be resolved automatically, direct the user to the appropriate escalation or support channel.
        """
        """
        C)  When responding:
            - Be clear, polite, and reassuring.
            - Include tracking information or estimated delivery times if available.
            - Offer next steps when needed (e.g., “We’ve contacted the courier…” or “You can update your address using…”).
                """)
    prompt = specialist_template("shipping",agent_prompt)
    chain = prompt | llm
    response = chain.invoke({
        "user_problem": state["user_problem"],
        "history": state["memory"].get("history", "")
    }).content
    return {"specialist_response": response}

In [67]:
# Summary agent with full context
def summary_agent(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        """
    A) You are a summary Agent responsible for summarizing the outcome of a support conversation and presenting it clearly to the user.
    B) You will receive:
        - The original user query.
        - The structured response from a specialist agent (Billing, Tech, or Shipping), including a summary message and status.

    C) Your task is to:
        - Review the original query and the summary provided.

    D) while giving summary take care of below points :
        - Clearly communicates the outcome.
        - Maintains a helpful and empathetic tone.
        - Offers next steps or contact options if the issue is unresolved.
        - Use plain language. Avoid jargon. Your goal is to leave the user feeling informed and supported.
    """
        """Full Conversation Context: {history}
        
        Original problem: {user_problem}
        Specialist response: {specialist_response}
        
        Craft a final response that:
        1. References previous interactions if relevant
        2. Incorporates the specialist's solution
        3. Provides clear next steps"""
    )
    
    chain = prompt | llm
    final_response = chain.invoke({
        "user_problem": state["user_problem"],
        "specialist_response": state["specialist_response"],
        "history": state["memory"].get("history", "")
    }).content
    # Update memory with this full exchange
    memory.save_context(
        {"input": state["user_problem"]},
        {"output": final_response}
    )
    
    return {
        "response": final_response,
        "memory": memory.load_memory_variables({})
    }
    

In [68]:
# Routing logic remains the same
def route_agent(state: State) -> str:
    if 'billing' in state['specialist_agent'].lower():
        return 'billing_agent'
    elif 'tech' in state['specialist_agent'].lower():
        return 'tech_agent'
    elif 'shipping' in state['specialist_agent'].lower():
        return 'shipping_agent'
    return "END"

In [69]:
# Build the workflow
workflow = StateGraph(State)
workflow.add_node('classify', problem_classification)
workflow.add_node('billing', billing_agent)
workflow.add_node('tech', tech_agent)
workflow.add_node('shipping', shipping_agent)
workflow.add_node('summarize', summary_agent)


In [70]:
workflow.set_entry_point('classify')

In [71]:
workflow.add_conditional_edges(
    'classify',
    route_agent,
    {
        "billing_agent": "billing",
        "tech_agent": "tech",
        "shipping_agent": "shipping"
    }
)
workflow.add_edge('billing', 'summarize')
workflow.add_edge('tech', 'summarize')
workflow.add_edge('shipping', 'summarize')
workflow.add_edge('summarize', END)

In [72]:
app = workflow.compile()

def run_conversation(query: str):
    # Load current memory state
    current_memory = memory.load_memory_variables({})
    
    # Execute the workflow
    results = app.invoke({
        "user_problem": query,
        "memory": current_memory
    })
    
    return {
        "response": results["response"],
        "memory": results["memory"]
    }

In [73]:
response1 = run_conversation("what should i do to buy a phone")
print(response1['response'])

KeyError: "Input to ChatPromptTemplate is missing variables {''}.  Expected: ['', 'history', 'user_problem'] Received: ['user_problem', 'history']\nNote: if you intended {} to be part of the string and not a variable, please escape it with double curly braces like: '{{}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "